# Neural Networks with TensorFlow
In this module, we're gonna be talking about how neural networks work and a little bit of the math behind them. Talking about gradient descent, back propagation, and how information flows to the neural network.

The whole point of neural network is to make a prediction or classification.  So, given some input information, then we feed it to the neural network, and then we want it to give us some output.